# Extract all DH sequences

In [1]:
def first(s):
    return(next(iter(s)))

In [2]:
from Bio import SeqIO

def find_DH_data(gb_file_name):
    DH_data = []
    KRDH = []
    module_CDS=[]
    for rec in SeqIO.parse(gb_file_name, 'gb'):
        for feature in rec.features:
            if feature.type == "CDS":
                types=first(feature.qualifiers)
#                 print(types)
                if types != None and types != 'gene_functions' and types != 'locus_tag': #Fjern None. Gem type
#                     print(feature.qualifiers) # Prints all CDS with actual clusters
#                     print(feature.qualifiers.keys()) # Prints all keys to the CDS dictionary
                    for i in range(len(feature.qualifiers[types])): 
                        if feature.qualifiers[types][i].find('PKS_DH') >=0 and feature.qualifiers[types][i].find('PKS_DHt') <=0: 
                            DH_data.append(feature.qualifiers[types][i])
                            module_CDS.append(feature.qualifiers['translation'])
#                             print(feature.qualifiers[types][i]) # Prints all DH domains
                        if 'PKS_DH' in feature.qualifiers[types][i] and 'PKS_DHt' not in feature.qualifiers[types][i]:
                            if 'PKS_KR' in feature.qualifiers[types][i-1]:
                                KRDH.append(True)
                            elif 'PKS_KR' not in feature.qualifiers[types][i-1]:
                                KRDH.append(None)               
    return DH_data, KRDH, module_CDS


def find_DH_seq(gb_file_name):
    sequences=[]
    DH_data=find_DH_data(gb_file_name)[0]
    KRDH = find_DH_data(gb_file_name)[1]
#     print(DH_data)
    for rec in SeqIO.parse(gb_file_name, 'gb'):
        for DH in DH_data:
            for feature in rec.features:
                if feature.type == "aSDomain":
                        if 'domain_id' in feature.qualifiers:
                            if DH.find(''.join(feature.qualifiers['domain_id'])) >= 0:
                                sequences.append(''.join(feature.qualifiers['translation']))
    return sequences,KRDH                           


def find_DH_seq_w_flanks(sequence, module_CDS):
    sequence_w_flanks=0
    if str(sequence) not in str(module_CDS):
        print('Error with sequence',sequence)    
    if str(sequence) in str(module_CDS):
        before_seq_after=str(module_CDS).partition(str(sequence)) # 3 tuple with the sequence before the DH seq, the DH seq, after the DH seq.
        if 'HPLL' in str(sequence) and 'HPALLD' in str(sequence):
#             print('Both in sequence') # print total sequence + 69 on the right side of HPALLD
            sequence_w_flanks=sequence
        elif 'HPLL' in str(sequence) and 'HPALLD' not in str(sequence):
#             print('HPLL in, but HPALLD not')
            sequence_w_flanks=sequence+before_seq_after[2][0:70]
        elif 'HPLL' not in str(sequence) and 'HPALLD' in str(sequence):
#             print('HPLL not in sequence, but HPALLD is')   
            sequence_w_flanks=before_seq_after[0][-40:-1]+sequence
        elif 'HPLL' not in str(sequence) and 'HPALLD' not in str(sequence):
            print('HPLL not in sequence nor is HPALLD') 
            sequence_w_flanks=before_seq_after[0][-40:-1]+sequence+before_seq_after[2][0:70]
    return sequence_w_flanks

# Print sekvenser med bestemt længde baseret på de fire udfald, inkoorporer det i resten af koden, og prøv at lave filerne + et alignment
# Dette vil give sekvenser, der er for lange, men måske er de nemmere at korte af end jeg tror
# Extra right side 69, extra left 25.
    
    
def find_HxxxGHxxxP(sequence):
    presence=None
    sequences=None
    sequence=str(sequence)
    for i in range(len(sequence)-9):
        if sequence[i] == 'H' and sequence[i+5] == 'H' and sequence[i+9] == 'P': #and sequence[i+4] == 'G': 
            sequences = sequence[i:i+9+1]
            presence = True
    return presence,sequences

In [111]:
test = find_DH_data(r'C:\Users\ASUS\Desktop\ny_bigscape_in\NCBI+nadine_no_dub\KYF85409.1.region001.gbk')
test2 = find_DH_seq(r'C:\Users\ASUS\Desktop\ny_bigscape_in\NCBI+nadine_no_dub\KYF85409.1.region001.gbk')
test3 = find_DH_seq_w_flanks(test2[0][0],test[2][0])
# print(test[2][0])
# print(test2[0][0])
print(test3)
print(len(test3))

RPGPLPPGPRSTPRGPHPLIDRVEAGALGPRAFHKRFSLRDWPVVEHRTQHAGLVSGTTLLEMARAAAALSGVGAVELHEVRFHAPCRVEDGAPQDVCLATERNGTRTEFSIFAPDPTGARERLASGYWVPALDEARERAPVESIRRRCRPAAARQLFEAVNASRLLRWGPRWDCIESLHLGEDEALADIRLRPEFAADLSVYGLHPALLDVATSLTAVLDPQGIYLPAGYGRLVCLASTPSRLFSHVRL
250


In [99]:
# result=str(test[2][0])
# result_part=result[-25:-1]
# print(result_part)
# print(result)

In [ ]:
## backup

# def find_DH_data(gb_file_name):
#     DH_data = []
#     KRDH = []
#     for rec in SeqIO.parse(gb_file_name, 'gb'):
#         for feature in rec.features:
#             if feature.type == "CDS":
#                 types=first(feature.qualifiers)
# #                 print(types)
#                 if types != None and types != 'gene_functions' and types != 'locus_tag': #Fjern None. Gem type
# #                     print(feature.qualifiers) # Prints all CDS with actual clusters
# #                     print(feature.qualifiers.keys()) # Prints all keys to the CDS dictionary
#                     for i in range(len(feature.qualifiers[types])): 
#                         if feature.qualifiers[types][i].find('PKS_DH') >=0 and feature.qualifiers[types][i].find('PKS_DHt') <=0: 
#                             DH_data.append(feature.qualifiers[types][i]) # Prints all DH domains
# #                             print(feature.qualifiers[types][i])
#                         if 'PKS_DH' in feature.qualifiers[types][i] and 'PKS_DHt' not in feature.qualifiers[types][i]:
#                             if 'PKS_KR' in feature.qualifiers[types][i-1]:
#                                 KRDH.append(True)
#                             elif 'PKS_KR' not in feature.qualifiers[types][i-1]:
#                                 KRDH.append(None)
                                
#     return DH_data, KRDH


# def find_DH_seq(gb_file_name):
#     sequences=[]
#     DH_data=find_DH_data(gb_file_name)[0]
#     KRDH = find_DH_data(gb_file_name)[1]
# #     print(DH_data)
#     for rec in SeqIO.parse(gb_file_name, 'gb'):
#         for DH in DH_data:
#             for feature in rec.features:
#                 if feature.type == "aSDomain":
#                         if 'domain_id' in feature.qualifiers:
#                             if DH.find(''.join(feature.qualifiers['domain_id'])) >= 0:
#                                 sequences.append(''.join(feature.qualifiers['translation']))
#     return sequences,KRDH     

In [3]:
def make_data_lst_of_lst(directory):
    lst_of_lst=[]
    import os
    for filename in os.listdir(directory):
        name=str(filename[:-4])
        gb_file_name=directory+'\{}'.format(filename)
        seq,KRDH=find_DH_seq(gb_file_name)#[0]
        module_CDS=find_DH_data(gb_file_name)[2]
        HGHP=[]
        HGHP_seq=[]
        for i in range(len(seq)):
            seq_w_flanks=find_DH_seq_w_flanks(seq[i],module_CDS[i])
            if find_HxxxGHxxxP(seq[i]):
                HGHP.append(find_HxxxGHxxxP(seq_w_flanks)[0])
                HGHP_seq.append(find_HxxxGHxxxP(seq_w_flanks)[1])
            else:
                HGHP.append(False)
                HGHP_seq.append(False)
            if any(name in sublist for sublist in lst_of_lst):
                name=name+'X'
            lst_of_lst.append([name,seq_w_flanks[0:264],KRDH[i],HGHP[i],HGHP_seq[i]])
    return lst_of_lst

tester=make_data_lst_of_lst(r'C:\Users\ASUS\Desktop\ny_bigscape_in\NCBI+nadine_no_dub')

HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence

HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence nor is HPALLD
HPLL not in sequence

In [4]:
import pandas as pd
df=pd.DataFrame(tester,columns=['name','AAseq','KRDH','HGHP','HGHP_seq'])

In [115]:
print(df)

                         name  \
0        AKF80291.1.region001   
1       AKF80291.1.region001X   
2      AKF80291.1.region001XX   
3        BAU88116.1.region001   
4       BAU88116.1.region001X   
..                        ...   
567  WP_159978968.1.region001   
568  WP_160164589.1.region001   
569  WP_160698423.1.region001   
570  WP_161860667.1.region001   
571  WP_164019660.1.region001   

                                                 AAseq  KRDH  HGHP    HGHP_seq  
0    ALRGAGARSTRGLLPGHPLLGMRRVEGEDLVYTQVMAPEQWVLDEH...  True  None        None  
1    WRAVTPVEGRQVSLPTYPWQRDRYWFQPGAAGASRSVGQHAVLGVS...  None  None        None  
2    YARRTVSLPTYPFQRQRYWWTREPRGQSALLAPREARAIPHLGSRL...  None  None        None  
3    ERPGRRTAHRGHPVLRERTVSALPPHAVHWSAPVDLTEFYLTDHRV...  None  None        None  
4    AAGLENRFGLDSGTPLTGHPLLRRLVQSTSTSRTYATVFTEESWIV...  True  True  HRIQEHGLVP  
..                                                 ...   ...   ...         ...  
567  AGDRATAPLGVGEAMDHPFLHAMHRLSPTEV

In [5]:
df.to_csv(r"C:\Users\ASUS\Desktop\ny_bigscape_in\data_from_ALL_DH_NCBI_dub=all_DH_unique_names_w_flanks.txt", sep="\t")

In [7]:
# import numpy as np
# df2=df
# filledgroups = df2.groupby("name")["KRDH","HGHP","HGHP_seq"].apply(lambda x:x.ffill().bfill()) # Makes new df 
# # filledgroups.to_csv(r"C:\Users\ASUS\Desktop\Extract_DH_domains\test1.txt", sep="\t")
# df2.loc[:,["KRDH","HGHP","HGHP_seq"]]=filledgroups.loc[:,["KRDH","HGHP","HGHP_seq"]]
# df2.drop_duplicates(subset=["name","KRDH","HGHP","HGHP_seq"], inplace=True)
# df2=df2.reset_index(drop=True)
# df2.to_csv(r"C:\Users\ASUS\Desktop\ny_bigscape_in\data_from_ALL_DH_NCBI_dub=HGHP_domain.txt", sep="\t")
# df3=df2.drop_duplicates(subset=["name"], keep='first')
# df3=df3.reset_index(drop=True)
# df3.to_csv(r"C:\Users\ASUS\Desktop\ny_bigscape_in\data_from_ALL_DH_NCBI_dub=NO.txt", sep="\t")

In [7]:
total=df.nunique()
ttotal=df2.nunique()
tttotal=df3.nunique()
print(total)
print(tttotal)

name        717
AAseq       514
KRDH          1
HGHP          1
HGHP_seq     22
dtype: int64
name        717
AAseq       514
KRDH          1
HGHP          1
HGHP_seq     22
dtype: int64


In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\ASUS\Desktop\ny_bigscape_in\data_from_ALL_DH_domains_mibig_unique_names_w_flanks.txt", sep="\t")
# df[df.all(axis='columns')]
df2=df.dropna()